In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
この実習で使うサンプルファイルをダウンロードします。
"""
!wget https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt -O mutmap_bulk.txt

# 大規模データ解析入門 (1)

## Contents

### Introduction
- [ゲノム解析とテキストデータ](#0.1)
- [テキストデータ処理の基本](#0.2)
- [今回のサンプルデータ](#0.3)

--- 

### Practice
1. [ファイルを開く/閉じる](#1.1)
1. [一行ずつ読む](#1.2)
1. [改行コードを取り除く](#1.3)
1. [一行を分割する](#1.4)
1. [SNP-indexを計算する](#1.5)
1. [ファイルに書き出す](#1.6)


---
## Introduction

### ゲノム解析とテキストデータ<a name="0.1"></a>

ゲノム解析を含め、大規模データ（ビッグデータ）の多くはテキストファイルです。  

例えば、「ゲノム配列」と言われるものは、DNAのの4つの塩基（アデニン、シトシン、グアニン、チミン）に相当するA、C、G、Tの文字が多量に並んだデータであり、その文字列がファイルに書かれています。  

テキストデータ中の多量の文字の中から、生物学的に意味のある箇所（データ）を見つけてくるのが、ゲノム解析といっても過言ではありません。

今回は、手元にある大規模データ（テキストファイル）をPythonで扱う方法を学んでいきます。

### テキストデータ処理の基本<a name="0.2"></a>

テキストデータ処理の基本は__「上の行から一行ずつ順番に処理する」__です。

その処理の流れは次の通りです。
- ファイルを開く
- 一行分のデータを取り出す
- 一行を分割する
- 一行のデータ処理をおこなう
- 次の行に移る
- ファイルを閉じる


### 今回のサンプルデータ=> [リンク](mutmap_bulk.txt)  <a name="0.3"></a>

MutMap (Abe et al., 2012)の論文のシーケンスを解析したデータを使います。  
MutMapとは、遺伝子マッピング手法の一つです（より詳しく知りたい方は、下記の論文をご覧ください）。  

Abe, A., Kosugi, S., Yoshida, K., Natsume, S., Takagi, H., Kanzaki, H., Matsumura, H., Yoshida, K., Mitsuoka, C., Tamiru, M., Innan, H., Cano, L., Kamoun, S., Terauchi, R. (2012). [Genome sequencing reveals agronomically important loci in rice using MutMap.](https://www.nature.com/articles/nbt.2095) _Nature biotechnology_, 30(2), 174.


MutMapの概要は次の通りです。
1. 突然変異系統とその元となった系統（オリジナル系統）を交配する。
1. 交配後第二世代（F2集団）を作成し、突然変異形質を持つ個体のDNAを集める（以下バルクDNA）。
1. オリジナル系統のDNAとバルクDNAをシーケンスする。
1. 両者のシーケンスを比較すると、突然変異の位置と、オリジナル系統の塩基や突然変異の塩基の種類がわかります。また、それぞれの塩基がいくつあるか（オリジナル系統と同じ塩基の個数と突然変異系統と同じ塩基の個数）といった情報も得られます。
1. 突然変異形質を持つF2個体は、ゲノム上に散在する変異のうち、形質に関わる変異を共通で持っているはずです。そのため、形質に関わる変異箇所では、突然変異系統の塩基のみ検出されることが期待されます。
1. そこで、変異箇所すべてについて、突然変異塩基の割合（「SNP-index」と呼ばれるもの）を調べます。
1. 調べた結果をグラフに描くことで、形質に関わる原因変異を特定または絞り込みができます。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/06/mutmap01.jpg?raw=true" alt="mutmap"></div>

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/06/mutmap02.png?raw=true" alt="mutmap"></div>

今回調べる形質は、イネの葉が「淡緑色（pale green）」となる形質です。  

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/06/mutmap04.jpg?raw=true" alt="mutmap"></div>


サンプルデータとして、この形質のMutMap解析のデータを扱います。  
データはテキストファイルです（ファイルの中身確認 => [リンク](mutmap_bulk.txt) ）。    
次のような情報が並んでいます。  

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/06/mutmap03.jpg?raw=true" alt="mutmap"></div>

それでは、葉を淡緑色にする原因の変異を見つけるために、
突然変異塩基の割合（SNP-index）が「1.0」になっているサイトを探しましょう。

---
## Practice

実習では、コードを少しずつ追加して、「SNP-indexを求めて、ファイルに出力するプログラム」を完成させていきます。

### 1. ファイルを開く/閉じる<a name="1.1"></a>
まず、ファイルを開きます。  
ついでに、ファイルを閉じる文も書きましょう。

=== 基本構文 ===
```python
f_in = open('ファイル名', 'r')
f_in.close()
```

=== 説明 ===
- `open()`
    - 第1引数: 開きたいファイル名
    - 第2引数:  
        * `'r'` (read-only mode) => 読み込み専用モードでファイルを開く。  
        * `'w'` (write mode) => 書き込みモードでファイルを開く。
        * `'a'` (append mode) => 追加書き込みモードでファイルを開く。
    - ファイル内容は 変数 `f_in` に保存している（変数はなんでも良い）。  
        * __ファイルオブジェクト__と呼ばれる状態で `f_in` に入っている
- `close()`
    - 使い方: ファイルオブジェクト.close()

In [ ]:
### 以下にプログラムを追加してください。 ###
dataset = 'mutmap_bulk.txt' # 読み込みファイル名


### 2. 一行ずつ読む<a name="1.2"></a>
開いたファイルを一行ずつ読み込み、出力してみましょう。

=== 基本構文 ===
```python
for line in f_in:
    print(line)
```

=== 説明 ===  

ファイルオブジェクト（変数 `f_in`）の中には、一行分が文字列がひとつの"データ"として入っています。  
`f_in` の中身のイメージ => [1行目の文字列, 2行目の文字列, 3行目の文字列, ..., 最終行の文字列]  

`for`文を使って、1行ずつ取り出すことができます。

In [ ]:
### 以下にプログラムを追加してください。 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名
f_in = open(dataset, 'r')  # ファイルを開く
f_in.close()  # ファイルを閉じる

### 3. 改行コードを取り除く<a name="1.3"></a>
上のコードでは、各行の間に空白行が含まれてしまっています。  
これは、ファイル内の各行の後ろ（行末）に、改行コード（不可視の特殊な文字のひとつ）が含まれているためです。  
解析によっては必須の処理ではありません。しかし、テキストファイル処理では、この改行コードを取り除くのが良いでしょう。  
（今回は、後のステップで行末にデータを追加するため、改行コードを除去しておく必要があります。）

=== 基本構文 ===  
```python
line = line.rstrip()
```

=== 説明 ===
- `rstrip()`
    - 使い方: 文字列オブジェクト.rstrip()
    - right-strip: 文字列やリストの一番右（行末）の文字を取り除く
    - ここでは`line`の行末の改行コード(`\n`)を取り除いています。
    
- `lstrip()`
    - 使い方: 文字列オブジェクト.lstrip()
    - left-strip: 文字列やリストの一番左（行頭）の文字を取り除く
    

In [ ]:
### 以下にプログラムを追加してください。 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名
f_in = open(dataset, 'r')  # ファイルを開く

# 1行ずつ処理する
for line in f_in:
    print(line)
    
f_in.close()  # ファイルを閉じる

### 4. 一行を分割する<a name="1.4"></a>
行内の各要素にアクセスするために、一行を分割しましょう。  
今回読み込んでいるファイルは「タブ文字」で要素間を区切ったファイル（__タブ区切りファイル__）であるため、`\t`(タブ文字)を指定して一行を分割します。  

=== 基本構文 ===  
```python
items = line.split('\t')
```

=== 説明 ===
- `split()`
    - 使い方: 文字列オブジェクト.split(区切り文字)
    - 区切り文字を指定して、文字列を分割し、リストに変換します。
        * 例えば、最初の行を分割すると、 `items`に `['chr10', '51406', 'G', 'A', '6', '3']`が入ります。
    - 読み込んだファイルがタブ区切りファイルのため、ここでは区切り文字にタブ文字`\t`を指定しています。
        * 主な区切り文字
            - `\t`: タブ
            - `,`: カンマ
    
   

In [1]:
### 以下にプログラムを追加してください。 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名
f_in = open(dataset, 'r')  # ファイルを開く

# 1行ずつ処理する
for line in f_in:
    
    line = line.rstrip()  # 行末の改行コードを除去する
    print(line)
    
f_in.close()  # ファイルを閉じる

chr10	51406	G	A	6	3
chr10	59101	A	T	6	3
chr10	59112	A	C	7	2
chr10	61001	A	T	11	3
chr10	161375	A	G	13	5
chr10	161561	A	C	6	5
chr10	161562	T	A	6	5
chr10	393574	A	G	8	2
chr10	465981	A	C	7	2
chr10	1076409	G	A	3	3
chr10	1076415	T	C	3	3
chr10	1330441	A	T	9	2
chr10	1435253	C	T	4	3
chr10	1435288	T	C	3	3
chr10	1544709	T	C	11	4
chr10	1567026	G	A	4	6
chr10	1613426	T	C	3	7
chr10	1613434	T	C	3	9
chr10	1613481	C	T	6	11
chr10	1982349	T	A	2	2
chr10	1982350	G	A	2	2
chr10	1982351	A	G	2	2
chr10	2010618	C	T	5	4
chr10	2246198	G	A	15	6
chr10	2443102	C	A	5	4
chr10	2470700	C	T	11	5
chr10	2470703	A	T	11	5
chr10	2755964	T	C	4	6
chr10	2757274	A	C	4	34
chr10	2757277	A	G	4	35
chr10	2797570	G	A	9	6
chr10	2939265	G	A	5	3
chr10	2979632	A	G	10	4
chr10	2979769	A	T	10	4
chr10	2979865	G	A	7	4
chr10	2979994	T	C	5	4
chr10	2981435	C	T	13	4
chr10	3094017	G	A	6	5
chr10	3120054	C	T	5	3
chr10	3165573	G	A	6	5
chr10	3165694	T	A	6	4
chr10	3209687	G	A	5	2
chr10	3248252	C	T	4	5
chr10	3390460	C	T	11	3
chr10	3390538	T	A	4	3
chr10	3391

### 5. SNP-indexを計算する<a name="1.5"></a>
SNP-indexを計算しましょう。以下の式で計算できます。

```python
SNP-index = ALTアリル数 / (REFアリル数 + ALTアリル数) 

REFアリル数は4番目要素
ALTアリル数は5番目要素
（* Pythonでは、0からカウントされることに注意してください。）
```

In [ ]:
### 以下にプログラムを追加してください。 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名

f_in   = open(dataset, 'r')  # ファイルを開く

# 1行ずつ処理する
for line in f_in:
    
    line = line.rstrip()  # 行末の改行コードを除去する
    items = line.split('\t')  # 行を分割する
    
    #print(line)
    print(items)
    
f_in.close()  # ファイルを閉じる

#### 補足事項: データ型の変換
REFアリル数とALTアリル数は文字列として`items`から取り出されます。  
数値計算するためには、それらを数値に変換する必要があります。   
そこで今回は`int()`を使って文字列を数値に変換しています。

=== 基本構文 ===  
```python
int(文字や小数点ありの数値)
```

=== 説明 ===
- `int()`
    - 使い方: int(数字のみの文字列)
    - 数字文字列や小数点ありの数値を整数値に変換する 
- `float()`
    -  小数点ありの数値に変換する
- `str()`
    - 文字列に変換する


### 6. ファイルに書き出す<a name="1.6"></a>
得られたSNP-indexを既存の文字列の行末に追加して、ファイルに書き出しましょう。

＊注意＊  
- 得られたSNP-indexを文字列として追加するためには、__float型（小数点あり数値）からstr型(文字列)に変換__する必要があります。  
- タブ区切りで出力するため、既存の文字列と追加するSNP-indexの間を`\t`で繋げる必要があります。
- 「3. 改行コードを取り除く」で改行コード`\n`を除去しているので、新たに改行コード`\n`を追加する必要があります。

=== 基本構文 ===
```python
f_out = open('ファイル名', 'w')
f_out.write(文字列)　
f_out.close()
```
=== 説明 ===
- write()
    - 使い方: 書き出し可能ファイルオブジェクト.write(文字列)
    - おもにファイルへの書き出しのために使われます。

In [ ]:
### 以下にプログラムを追加してください。 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名

f_in   = open(dataset, 'r')  # ファイルを開く

# 1行ずつ処理する
for line in f_in:
    
    line = line.rstrip()  # 行末の改行コードを除去する
    items = line.split('\t')  # 行を分割する
    snpindex = int(items[5]) / (int(items[4]) + int(items[5]))  # SNP-indexを計算する

    #print(line)
    #print(items)
    #print(items[1])
    print(snpindex)    
    
f_in.close()  # ファイルを閉じる

#### 補足事項: 出力する文字列の中に変数を埋め込む方法
出力する文字列の中に変数の値を埋め込みたい場合、次のようなコードを書きます。  

```python
x = 5.3
y = 3.4

print('%d + %d = %d' % (x, y, x + y))
# 出力結果 => 5 + 3 = 8

print('%f + %.3f = %.2f' % (x, y, x + y))
# 出力結果 => 5.300000 + 3.400 = 8.70

print('%s + %s = %s' % (x, y, x + y))
# 出力結果 => 5.3 + 3.4 = 8.7
```
`print()`内は、`%`を境に2つの部品に分かれます。  
例えば、`'%d + %d = %d'`　と `(x, y, x + y)`

前半の部品 `'%d + %d = %d'` は出力する文字列骨格です。  
文字列骨格の中に`%d`が3つあります。  
それぞれの`%d`に後半の部品 `(x, y, x + y)` の値が前から順番に代入されます。  

また、`%d`や`%f`、`%s`で代入される値の型も指定できます。
- `%d`: 整数値（integer）。小数点がある場合は切り捨てです。
- `%f`: 浮動小数点値（float）。さらに小数点の桁数も指定できます(例えば、`%.3f`は小数点第3位まで表示)。
- `%s`: 文字列（string）。数値は強制的に文字列として扱われます。

In [ ]:
x = 5.3
y = 3.4

print('%d + %d = %d' % (x, y, x + y))
# 出力結果 => 5 + 3 = 8

print('%f + %.3f = %.2f' % (x, y, x + y))
# 出力結果 => 5.300000 + 3.400 = 8.70

print('%s + %s = %s' % (x, y, x + y))
# 出力結果 => 5.3 + 3.4 = 8.7

### 完成したプログラム


In [ ]:
### 完成形 ###

dataset = 'mutmap_bulk.txt' # 読み込みファイル名
outdata = 'mutmap_snpindex.txt' # 書き出しファイル名

f_in   = open(dataset, 'r')  # ファイルを開く
f_out = open(outdata, 'w')  # ファイルを開く

# 1行ずつ処理する
for line in f_in:
    
    line = line.rstrip()  # 行末の改行コードを除去する
    items = line.split('\t')  # 行を分割する
    snpindex = int(items[5]) / (int(items[4]) + int(items[5]))  # SNP-indexを計算する
    out_line = line + '\t' + str(snpindex)  # 行末（リストの最後）にSNP-indexデータを追加する
                                                                    # タブ区切りで出力させるために、既存の文字列とSNP-indexの間をタブ文字で繋げている
    #print(line)
    #print(items)
    #print(items[1])
    #print(snpindex)    
    print(out_line)
    
    f_out.write('%s\n' % out_line)  # ファイルに書き出し（行末に改行コードを追加する）
    
f_in.close()  # ファイルを閉じる
f_out.close()

今回実習した方法は、非常に適用範囲の広く、ほとんどのテキストファイル処理に使える方法です。

その一方で、次のような短所もあります。
- 処理速度が遅い
- プログラムが長くなる

テーブル状のデータの場合、「Pandas」と呼ばれるライブラリを使うことで、より簡単(?)に扱えるようになります。  
次回、同じデータを「Pandas」で処理してみて、その便利さを体験してもらう予定です。